# Collection Deletion

Firestore keeps subcollections even when a parent document is removed.
Fire-Prox provides helpers to clear an entire collection (and optionally
its nested subcollections) using batched deletes that work with the emulator
and production projects alike.

You can inspect available subcollections with `doc.collections()` or `db.collections("path/to/doc")`,
and document deletes now clean up descendants recursively by default (pass `recursive=False` to skip).


In [ ]:
from google.cloud import firestore
from fire_prox import FireProx

client = firestore.Client(project="my-project")
db = FireProx(client)

# Delete everything in 'users', including nested subcollections
users = db.collection('users')
summary = users.delete_all(batch_size=100, recursive=True)
print(summary)  # {'documents': 42, 'collections': 7}

# Delete just a single nested subcollection
user = db.doc('users/alovelace')
posts_summary = user.delete_subcollection('posts')
print(posts_summary)

# Inspect subcollections from document or FireProx
print(user.collections(names_only=True))
print(db.collections("users/alovelace", names_only=True))

# Recursive document delete (default behaviour)
user.delete()

# Skip recursion when needed
grace = db.doc("users/gracehopper")
grace.delete(recursive=False)


In [ ]:
import asyncio
from google.cloud import firestore_async
from fire_prox import AsyncFireProx

async def main():
    client = firestore_async.AsyncClient(project="my-project")
    db = AsyncFireProx(client)

    users = db.collection('users')
    summary = await users.delete_all(batch_size=50)
    print(summary)

    user = db.doc('users/bob')
    comments = await user.delete_subcollection('comments', dry_run=True)
    print(comments)  # Dry-run counts only

    print(await user.collections(names_only=True))
    print(await db.collections("users/bob", names_only=True))

    # Recursive delete (default)
    await user.delete()

    # Non-recursive delete
    other = db.doc("users/gracehopper")
    await other.delete(recursive=False)

asyncio.run(main())
